# Clase 1.2 - Encapsulamiento y Propiedades en Python

**Unidad:** 1 - POO Avanzada con Python  
**Conexión al Proyecto TaskFlow:** Aprenderemos a proteger y validar los datos de nuestros modelos Usuario, Proyecto y Tarea.  

## 📋 Objetivos de Aprendizaje

Al finalizar esta clase, serás capaz de:
- [ ] Comprender las convenciones `_` y `__` en Python
- [ ] Usar `@property` para crear getters pythonicos
- [ ] Implementar setters con `@nombre.setter`
- [ ] Aplicar `@dataclass` para clases más limpias
- [ ] Usar `__post_init__` para validaciones en dataclasses

## 💻 Conexión con TaskFlow

En el sistema **TaskFlow**, necesitamos:
- Validar que el email tenga formato correcto
- Asegurar que el username sea único
- Calcular valores derivados (como dominio del email)
- Proteger datos sensibles (como password_hash)

In [ ]:
# Importaciones
from dataclasses import dataclass, field
from datetime import datetime
from typing import Optional
import re

---

## 1. 🔒 Convenciones de Encapsulamiento en Python

**📐 Diferencia clave:** Python no tiene modificadores de acceso privados reales (como `private` en Java). En su lugar, usa **convenciones de nombres**.

| Prefijo | Significado | Accesibilidad | Uso típico |
|---------|------------|----------------|------------|
| `sin_prefijo` | Público | 🚫 Accesible desde cualquier lugar | Atributos y métodos normales |
| `_` (un guion) | Protegido | ⚠️ Convención: "no usar directamente" | API interna, atributos sensibles |
| `__` (doble guion) | Privado (name-mangling) | 🔒 Difícil acceder externamente | Evitar colisiones de nombres |

**💡 Importante:** Estas son **convenciones**, no barreras reales. Python confía en el desarrollador ("we're all adults here").

In [ ]:
# Ejemplo 1: Convenciones de encapsulamiento

class Usuario:
    """Representa un usuario en TaskFlow."""
    
    def __init__(self, username: str, email: str, password: str):
        # Público: accesible directamente
        self.username = username
        self.email = email
        
        # Protegido (_): convención: no modificar directamente
        self._password_hash = self._hash_password(password)
        
        # Privado (__): name-mangling, difícil de acceder
        self.__session_token = self._generate_token()
    
    def _hash_password(self, password: str) -> str:
        """Método protegido: hashea el password (simplificado)."""
        return f"HASH_{password}"  # En producción: usar bcrypt
    
    def _generate_token(self) -> str:
        """Método privado: genera token de sesión."""
        return f"TOKEN_{datetime.now().timestamp()}"
    
    def verificar_password(self, password: str) -> bool:
        """Verifica si el password es correcto."""
        return self._password_hash == self._hash_password(password)

# Crear usuario
u = Usuario("jdoe", "john@example.com", "secreto123")

# Acceso a atributos públicos
print(f"Público: {u.username}")

# Acceso a atributo protegido (posible, pero no recomendado)
print(f"Protegido: {u._password_hash}")  # ⚠️ Funciona, pero es convención no hacerlo

# Intentar acceder a atributo privado
try:
    print(u.__session_token)
except AttributeError as e:
    print(f"Error: {e}")

# Acceder con name-mangling (no recomendado, pero posible)
print(f"Privado (con name-mangling): {u._Usuario__session_token}")

**💡 Nota:** El **name-mangling** transforma `__atributo` a `_NombreClase__atributo` para evitar colisiones en herencia. No es verdadera privacidad, solo dificulta el acceso.

---

## 2. 📋 Property: Getters Pythonicos

**📐 Definición:** `@property` es un **decorador** que permite convertir un método en un atributo calculado, sin necesidad de paréntesis.

**📋 Ventajas:**
- Calcula valores al vuelo (no los almacena)
- Permite agregar lógica antes de retornar
- Mantiene la interfaz limpia (sin paréntesis)

**❌ Enfoque NO pythonico (Java-style):**
```python
def get_username(self):
    return self._username
```

**✅ Enfoque pythonico:**
```python
@property
def username(self):
    return self._username
```

In [ ]:
# Ejemplo 2: Usando @property

class Usuario:
    """Representa un usuario en TaskFlow."""
    
    def __init__(self, username: str, email: str):
        self._username = username  # Atributo "privado"
        self._email = email
    
    @property
    def username(self) -> str:
        """Retorna el nombre de usuario."""
        return self._username
    
    @property
    def email(self) -> str:
        """Retorna el email."""
        return self._email
    
    @property
    def dominio_email(self) -> str:
        """
        Propiedad calculada: extrae el dominio del email.
        No se almacena, se calcula al vuelo.
        """
        if "@" in self._email:
            return self._email.split("@")[1]
        return ""
    
    @property
    def es_email_corporativo(self) -> bool:
        """Verifica si es un email corporativo."""
        dominios_personales = ["gmail.com", "yahoo.com", "hotmail.com"]
        return self.dominio_email not in dominios_personales

# Crear usuario
u = Usuario("jdoe", "john@company.com")

# Acceder a propiedades (sin paréntesis)
print(f"Username: {u.username}")
print(f"Email: {u.email}")
print(f"Dominio: {u.dominio_email}")
print(f"¿Es corporativo? {u.es_email_corporativo}")

# Intentar modificar propiedad (sin setter) genera error
try:
    u.username = "nuevo_username"
except AttributeError as e:
    print(f"\nError (esperado): {e}")

**💡 Nota:** Sin un `@username.setter`, la propiedad es **read-only** (solo lectura).

---

## 3. ⚙︎ Setters: Validación de Datos

**📐 Definición:** Un **setter** permite validar y controlar la modificación de un atributo. Se crea con `@nombre_propiedad.setter`.

**📋 Patrón completo:**
```python
@property
def email(self):
    return self._email

@email.setter
def email(self, nuevo_email):
    # Validaciones aquí
    self._email = nuevo_email
```

In [ ]:
# Ejemplo 3: Setters con validación

class Usuario:
    """Representa un usuario en TaskFlow con validaciones."""
    
    def __init__(self, username: str, email: str):
        # Usar los setters para validación en la inicialización
        self.username = username
        self.email = email
    
    @property
    def username(self) -> str:
        """Retorna el nombre de usuario."""
        return self._username
    
    @username.setter
    def username(self, valor: str):
        """
        Establece el nombre de usuario con validaciones.
        
        Raises:
            ValueError: Si el username es inválido
        """
        if not isinstance(valor, str):
            raise TypeError("Username debe ser un string")
        if len(valor) < 3:
            raise ValueError("Username debe tener al menos 3 caracteres")
        if not valor.isalnum():
            raise ValueError("Username debe ser alfanumérico")
        self._username = valor
    
    @property
    def email(self) -> str:
        """Retorna el email."""
        return self._email
    
    @email.setter
    def email(self, valor: str):
        """
        Establece el email con validaciones.
        
        Raises:
            ValueError: Si el email es inválido
        """
        if "@" not in valor or "." not in valor:
            raise ValueError("Email debe contener @ y .")
        self._email = valor

# Crear usuario con datos válidos
u = Usuario("jdoe", "john@example.com")
print(f"Usuario creado: {u.username} - {u.email}")

# Modificar username válido
u.username = "johndoe"
print(f"Username modificado: {u.username}")

# Intentar username inválido
try:
    u.username = "ab"  # Demasiado corto
except ValueError as e:
    print(f"\nError de validación (esperado): {e}")

# Intentar email inválido
try:
    u.email = "invalido"  # Sin @ ni .
except ValueError as e:
    print(f"Error de validación (esperado): {e}")

**💡 Ventaja:** Al usar los setters en `__init__`, nos aseguramos de que las validaciones se apliquen siempre, incluso al crear el objeto.

---

## 4. 🔫 Dataclasses: Clases Más Limpias

**📐 Definición:** `@dataclass` es un decorador que **genera automáticamente**:
- `__init__`
- `__repr__`
- `__eq__`
- Y más métodos

**📋 Ventajas:**
- Menos código repetitivo
- Menos propenso a errores
- Más fácil de leer

**❌ Sin dataclass:**
```python
class Usuario:
    def __init__(self, username, email):
        self.username = username
        self.email = email
    def __repr__(self):
        return f"Usuario({self.username})"
    def __eq__(self, other):
        return self.username == other.username
```

**✅ Con dataclass:**
```python
@dataclass
class Usuario:
    username: str
    email: str
```

In [ ]:
# Ejemplo 4: Uso básico de dataclass

from dataclasses import dataclass

@dataclass
class Usuario:
    """Representa un usuario en TaskFlow."""
    username: str
    email: str
    nombre_completo: str = ""  # Valor por defecto

# Crear usuarios
u1 = Usuario("jdoe", "john@example.com", "John Doe")
u2 = Usuario("jdoe", "john@example.com", "John Doe")
u3 = Usuario("asmith", "alice@example.com")

# __repr__ automático
print(f"Representación: {u1}")

# __eq__ automático (compara todos los campos)
print(f"\nu1 == u2: {u1 == u2}")  # True (mismos valores)
print(f"u1 == u3: {u1 == u3}")  # False (valores diferentes)

# Acceso a atributos
print(f"\nUsername: {u1.username}")
print(f"Nombre completo: {u1.nombre_completo}")

**💡 Nota:** Los campos sin valor por defecto deben ir **antes** de los campos con valor por defecto.

---

## 5. 🔄 Validaciones con `__post_init__`

**📐 Definición:** `__post_init__` es un método especial que se ejecuta **después** de `__init__` en las dataclasses. Es ideal para validaciones y cálculos derivados.

**📋 Para habilitarlo:**
```python
@dataclass
class MiClase:
    campo: str = field(init=False)  # No se pasa en __init__
    
    def __post_init__(self):
        # Validaciones y cálculos aquí
```

In [ ]:
# Ejemplo 5: __post_init__ para validaciones

@dataclass
class Usuario:
    """Usuario de TaskFlow con validaciones."""
    username: str
    email: str
    nombre_completo: str = ""
    
    # Campos calculados (no se pasan al crear)
    creado_en: datetime = field(init=False, default_factory=datetime.now)
    es_valido: bool = field(init=False, default=False)
    errores: list[str] = field(init=False, default_factory=list)
    
    def __post_init__(self):
        """Valida el usuario después de la inicialización."""
        # Validar username
        if len(self.username) < 3:
            self.errores.append("Username debe tener al menos 3 caracteres")
        
        # Validar email
        if "@" not in self.email:
            self.errores.append("Email debe contener @")
        
        # Marcar como válido si no hay errores
        self.es_valido = len(self.errores) == 0
    
    @property
    def dominio_email(self) -> str:
        """Extrae el dominio del email."""
        return self.email.split("@")[1] if "@" in self.email else ""

# Crear usuarios
u1 = Usuario("jdoe", "john@example.com", "John Doe")
print(f"Usuario válido: {u1}")
print(f"¿Es válido? {u1.es_valido}")
print(f"Errores: {u1.errores}")
print(f"Dominio: {u1.dominio_email}")

u2 = Usuario("ab", "invalido", "Jane Doe")
print(f"\nUsuario inválido: {u2}")
print(f"¿Es válido? {u2.es_valido}")
print(f"Errores: {u2.errores}")

**💡 Ventaja:** Las validaciones se ejecutan automáticamente al crear el objeto, sin necesidad de llamar un método explícitamente.

---

## 6. 📋 Comparativa: Clásica vs Dataclass

| Aspecto | Clásica | Dataclass |
|---------|---------|-----------|
| **Código** | Más verboso | Más conciso |
| **`__init__`** | Manual | Automático |
| **`__repr__`** | Manual | Automático |
| **`__eq__`** | Manual | Automático |
| **Validaciones** | En `__init__` o setters | En `__post_init__` |
| **Properties** | Funcionan igual | Funcionan igual |
| **Type hints** | Opcionales | Recomendados |

In [ ]:
# Ejemplo 6: Clase Proyecto con dataclass + properties

@dataclass
class Proyecto:
    """Proyecto en TaskFlow."""
    nombre: str
    descripcion: str
    horas_estimadas: int = 0
    horas_reales: int = 0
    
    creado_en: datetime = field(init=False, default_factory=datetime.now)
    
    @property
    def excede_tiempo(self) -> bool:
        """Verifica si el proyecto excedió el tiempo estimado."""
        if self.horas_estimadas == 0:
            return False
        return self.horas_reales > self.horas_estimadas
    
    @property
    def porcentaje_completado(self) -> float:
        """
        Calcula el porcentaje de tiempo usado.
        
        Returns:
            Porcentaje (0-100+), None si no hay estimación
        """
        if self.horas_estimadas == 0:
            return None
        return (self.horas_reales / self.horas_estimadas) * 100
    
    @property
    def estado_tiempo(self) -> str:
        """Retorna una descripción del estado del tiempo."""
        pct = self.porcentaje_completado
        if pct is None:
            return "Sin estimar"
        if pct > 120:
            return "🟠 Crítico (excede 120% del tiempo)"
        elif pct > 100:
            return "🔴 Excede tiempo estimado"
        elif pct > 80:
            return "🟡 Cerca del límite"
        else:
            return "🟢 En tiempo"

# Crear proyectos
p1 = Proyecto("TaskFlow", "Sistema de gestión", 40, 35)
print(f"Proyecto: {p1.nombre}")
print(f"Estado: {p1.estado_tiempo}")
print(f"Porcentaje: {p1.porcentaje_completado:.1f}%")

p2 = Proyecto("App Móvil", "Aplicación Android", 80, 100)
print(f"\nProyecto: {p2.nombre}")
print(f"Estado: {p2.estado_tiempo}")
print(f"¿Excede tiempo? {p2.excede_tiempo}")

---

## 📝 Ejercicio Práctico: Tarea con Validaciones

Crea una dataclass `Tarea` con:
- `titulo`: str (requerido)
- `descripcion`: str (requerido)
- `prioridad`: int (1= baja, 2= media, 3= alta, por defecto 2)
- `estado`: str (por defecto "pendiente")

Validaciones en `__post_init__`:
- `titulo` no puede estar vacío
- `prioridad` debe estar entre 1 y 3
- `estado` debe ser uno de: "pendiente", "en_progreso", "completada"

Properties:
- `es_alta_prioridad`: bool (True si prioridad == 3)
- `esta_completada`: bool (True si estado == "completada")

In [ ]:
# Tu solución aquí
from dataclasses import dataclass, field
from typing import Literal

@dataclass
class Tarea:
    """Representa una tarea en TaskFlow."""
    
    # Campos
    titulo: str
    descripcion: str
    prioridad: int = 2
    estado: str = "pendiente"
    
    # Campos calculados
    errores: list[str] = field(init=False, default_factory=list)
    
    def __post_init__(self):
        """Valida la tarea después de la inicialización."""
        # TODO: Implementar validaciones
        pass
    
    @property
    def es_alta_prioridad(self) -> bool:
        """Retorna True si la tarea es de alta prioridad."""
        # TODO: Completar
        pass
    
    @property
    def esta_completada(self) -> bool:
        """Retorna True si la tarea está completada."""
        # TODO: Completar
        pass

### ✅ Validación Automática

In [ ]:
# Tests automáticos

def test_tarea():
    """Ejecuta tests de la clase Tarea."""
    
    # Test 1: Tarea válida
    t1 = Tarea("Implementar auth", "Crear sistema de login")
    assert len(t1.errores) == 0, f"Test 1 falló: {t1.errores}"
    assert not t1.es_alta_prioridad
    assert not t1.esta_completada
    print("✅ Test 1: Tarea válida creada")
    
    # Test 2: Tarea de alta prioridad
    t2 = Tarea("Fix bug", "Error crítico", prioridad=3)
    assert t2.es_alta_prioridad
    print("✅ Test 2: Tarea de alta prioridad")
    
    # Test 3: Tarea completada
    t3 = Tarea("Diseñar UI", "Crear mockups", estado="completada")
    assert t3.esta_completada
    print("✅ Test 3: Tarea completada")
    
    # Test 4: Título vacío inválido
    t4 = Tarea("", "Descripción")
    assert len(t4.errores) > 0
    assert any("titulo" in err.lower() or "vac" in err.lower() for err in t4.errores)
    print("✅ Test 4: Título vacío detectado")
    
    # Test 5: Prioridad inválida
    t5 = Tarea("Test", "Desc", prioridad=5)
    assert len(t5.errores) > 0
    assert any("prioridad" in err.lower() for err in t5.errores)
    print("✅ Test 5: Prioridad inválida detectada")
    
    print("\n🚀 ¡Todos los tests pasaron!")

# Ejecutar tests
try:
    test_tarea()
except AssertionError as e:
    print(f"\n❌ {e}")
    print("\nRevisa tu implementación.")

### 📋 Solución del Ejercicio

In [ ]:
# Solución completa

@dataclass
class Tarea:
    """Representa una tarea en TaskFlow."""
    
    titulo: str
    descripcion: str
    prioridad: int = 2
    estado: str = "pendiente"
    
    errores: list[str] = field(init=False, default_factory=list)
    
    ESTADOS_VALIDOS = ["pendiente", "en_progreso", "completada"]
    
    def __post_init__(self):
        """Valida la tarea después de la inicialización."""
        # Validar título
        if not self.titulo or self.titulo.strip() == "":
            self.errores.append("El título no puede estar vacío")
        
        # Validar prioridad
        if self.prioridad not in [1, 2, 3]:
            self.errores.append("Prioridad debe ser 1, 2 o 3")
        
        # Validar estado
        if self.estado not in self.ESTADOS_VALIDOS:
            self.errores.append(f"Estado debe ser uno de: {', '.join(self.ESTADOS_VALIDOS)}")
    
    @property
    def es_alta_prioridad(self) -> bool:
        """Retorna True si la tarea es de alta prioridad."""
        return self.prioridad == 3
    
    @property
    def esta_completada(self) -> bool:
        """Retorna True si la tarea está completada."""
        return self.estado == "completada"

# Ejecutar tests de nuevo
test_tarea()

---

## 📈 Diagrama: Ciclo de Vida de una Dataclass

```mermaid
graph TD
    A[Crear instancia Tarea(...)] --> B[__init__ automático]
    B --> C[__post_init__ se ejecuta]
    C --> D[Validaciones]
    D --> E{¿Hay errores?}
    E -->|Sí| F[Campos de error poblados]
    E -->|No| G[Objeto válido]
    F --> H[Objeto listo]
    G --> H
    
    style A fill:#e1f5ff
    style C fill:#fff9c4
    style D fill:#ffccbc
    style E fill:#c8e6c9
    style H fill:#d1c4e9
```

---

## 📝 Resumen de la Clase

### 📋 Conceptos Clave

| Concepto | Propósito | Ejemplo |
|----------|-----------|----------|
| `_atributo` | Convención: protegido | `self._password` |
| `__atributo` | Name-mangling: "privado" | `self.__token` |
| `@property` | Getter pythonico | `@property def email(self):` |
| `@x.setter` | Setter con validación | `@email.setter def email(self, val):` |
| `@dataclass` | Clases más limpias | `@dataclass class Usuario:` |
| `__post_init__` | Validaciones en dataclass | `def __post_init__(self):` |

### 📋 Buenas Prácticas

1. **Usar `_` para atributos internos** que no deberían modificarse directamente
2. **Usar `@property` sin setter** para atributos read-only
3. **Usar `@dataclass`** para clases que principalmente almacenan datos
4. **Validar en `__post_init__`** en lugar de sobrescribir `__init__`
5. **Combinar dataclass con properties** para lo mejor de ambos mundos

### 🤝 Conexión con TaskFlow

Hemos aprendido a:
- Proteger el password con `_password_hash`
- Validar email con `@email.setter`
- Crear modelos limpios con `@dataclass`
- Calcular valores derivados con `@property`

**📈 Próxima clase:** Herencia y Composición

Aprenderemos a:
- Crear jerarquías de clases
- Usar `super()` correctamente
- Aplicar composición vs herencia
- Diseñar el modelo base de TaskFlow